# Thanks to 

> KoalaBear84(https://github.com/KoalaBear84, https://www.reddit.com/user/KoalaBear84/)
for creating OpenDirectoryDownloader

> Jasimtanur (https://www.reddit.com/user/jasimtanur/)
for creating code for both Installing and Indexing section


# To Up-Date


  >Thumbnail Generator

  >Gif Generator

  >Better Filter Function

  

# Read Before Using this Notebook


---


**Google Drive Indexing and Downloading Support Not Implemented Yet..**

---



**Notebook Specification** : 

---


> Disk Storage : 107 GB (Free 77GB)

> RAM Memory   : 12 GB

> Life Time    : 12 Hours (For Pro users 24 Hours)

> Click on "connect" to connect to Virtual Machine.It can be done by many ways also

> Left side File manager show what is inside "/content/" Folder

For Google Drive Free Users:

---

> Maximum Drive space is 15GB, So only OD less than 14GB can be saved to Drive

> To Download necessary files filter the links in URL file 

> Copy URL file to Google drive so that you can resume Downloading OD .Each time you run this notebook read Links from URL file located inside google drive


For Google Drive Premium Users (100GB, 200GB, Unlimited) :

---


> You can download most of the OD (based on size of these ODs)


## Why I Created Colab Notebook

> I created this notebook because the place I currenly living have issues with frequent power outage, poor tower signal (Very slow internet 1MB/Sec).

> I cannot index or download due to this issues.

> Thats is why I created a Colab notebook to automate downloading

> With this notebook I can index OD and filter URL files, So that only necessary files will be archived and will be saved to Google Drive

> I can download archives from Google Drive quickly and with pause and resume support



## Usage :

> Run each cell one by one by order.

> Run only one instance of OD at a time.

> If you want to run 2 or more OD then create copy of this notebook and run each.

# Mount Drive

Usage :

        0.Run below cell 
        1.Click on the link
        2.Select which Google drive account to store
        3.Click on "Allow"
        4.Copy the code
        5.Paste the code in the rectangular box

In [ ]:
# Mounting Google Drive to this Virtual Machine
from google.colab import drive
drive.mount('/content/drive')

# Installing dotnet and ODD-KB (Credit : Jasimtanur)

In [ ]:
%%capture
# Package to enable file management
import os

if not os.path.exists("/content/OpenDirectoryDownloader-linux-x64.zip"):
  # Installing Microsoft product
  !wget https://packages.microsoft.com/config/ubuntu/20.04/packages-microsoft-prod.deb -O packages-microsoft-prod.deb
  !sudo dpkg -i packages-microsoft-prod.deb

  # Updating Linux Environment
  !sudo apt-get update; \
    apt-get install -y apt-transport-https && \
    apt-get update && \
    apt-get install -y dotnet-sdk-3.1
  !sudo apt-get update; \
    sudo apt-get install -y apt-transport-https && \
    sudo apt-get update && \
    sudo apt-get install -y aspnetcore-runtime-3.1 

  # Installing Dotnet
  !sudo apt-get install -y dotnet-runtime-3.1

  # Downloading OpenDirectoryDownloader-KoalaBear84 (ODD-KB): Version 1.6.0.2
  !wget https://github.com/KoalaBear84/OpenDirectoryDownloader/releases/download/v1.6.0.2/OpenDirectoryDownloader-linux-x64.zip

  # Extracting ODD-KB
  !unzip /content/OpenDirectoryDownloader-linux-x64.zip -d /content/Indexer/
else:
  print('Necessary programs are installed already')

# Indexing (KoalaBear84 & Jasimtanur)

In [ ]:
#@markdown <h2>Lets Index the Open Directory</h2>

OD_URL = "www" #@param {type:"string"}
threads = "20" #@param {type:"string"}
OD_title = "www" #@param {type:"string"}

if OD_URL != '' and threads != '' and OD_title != '' :
  arguments = '-u {} -t {} -q -j -s | tee "{}.log"'.format(OD_URL, threads, OD_title)
  !dotnet /content/Indexer/OpenDirectoryDownloader.dll $arguments
else:
  print('Enter all values')

In [ ]:
# Displaying Result Markdown
with open('{}.log'.format(OD_title), 'r')as f1:
  output = f1.read()

if 'URLs with errors:' in output:
  output = '\n'.join(output[ output.rfind('|**Url:**'):output.rfind('URLs with errors:')-2].splitlines())
else:
  output = '\n'.join(output[ output.rfind('|**Url:**'):output.rfind('Save session to JSON')-2].splitlines())

from IPython.display import display, Markdown as md
display(md(output))

# Threaded Download Initiator

In [ ]:
# Importing Necessary Packages
from multiprocessing.dummy import Pool
from multiprocessing import Pool as Pool_mp
from tqdm.notebook import tqdm
from datetime import datetime as dati
from pytz import timezone
import os
import json
from google.colab import output as cell_output
from threading import Thread
import time
import sys
from PIL import Image

from urllib.parse import quote, unquote

# Function to return file size
def file_size(i):
  n, p, d = 0, 1024, ['Bytes', 'KB', 'MB', 'GB', 'TB']
  while i > p:
    i /= p
    n += 1
  return '{:.2f} {}'.format(i, d[n])

# Function to calculate a folder size
def get_folder_size(folder):
  total_size = 0
  all_files = []
  for path, dirs, files in os.walk(folder):
    for file in files:
      fp = os.path.join(path, file)
      all_files.append(fp)
      total_size += os.path.getsize(fp)
  # print('Folder Name : {}\nFolder size : {}\nNumber of files : {}'.format(folder, file_size(total_size), len(all_files)))
  return [folder, total_size, len(all_files)]

# Function to calculate a folder size
def show_folder_size(folder):
  i = get_folder_size(folder)
  print('Folder Name : {}\nFolder size : {}\nNumber of files : {}'.format(i[0], file_size(i[1]), i[2]))

# Function to download files with Wget
# def threaded_download_initiator_wget(i):
#   !wget "$i" -nc -bqx -P "Temp" >/dev/null 2>&1
#   tqdm_bar.update(1)
#   return 0

# Function to download files with Wget
def threaded_download_initiator(i):
  j = 0
  size, url = i[0], i[1]
  !wget "$url" -nc -x -P "Temp" >/dev/null 2>&1
  tqdm_bar.update(1)
  # print(j)
  # j += 1
  downloaded_size[0] += size
  # print(j)
  # j += 1
  tqdm_bar_size.set_description_str('{:.1f}/{} MB: '.format(downloaded_size[0]/mb_size[0], total_mb_size[0]))
  # print(j)
  # j += 1
  tqdm_bar_size.update(size)
  # print(j)
  # j += 1
  return 0

# Function to start thread
def start_thread(target, number_of_threads):
  # Creating number of threads 
  pool1 = Pool(number_of_threads)
  # Running created threads
  _ = pool1.map_async(threaded_download_initiator,target)
  # Closing threads
  pool1.close()
  pool1.join()

# Find all file types in json with it's size and count
def find_filetypes(json_data):
  popular_file_types = {}
  for i in json_data:
    temp_ext = os.path.splitext(i[1])[1].lower()
    temp_size = i[0]

    if temp_ext not in popular_file_types:
      popular_file_types[temp_ext] = {'Count' : 1, 'Size' : temp_size}
    else:
      popular_file_types[temp_ext]['Count'] += 1
      popular_file_types[temp_ext]['Size'] += temp_size
  return popular_file_types

# Function to show files types info
def show_filestypes(file_types):
  file_types = sorted([[i].extend(file_types[i]) for i in file_types], key=lambda x: x[2], reverse=True)

  for i in file_types:
    output_lines.append('{:6s} - {:10s} - {}'.format(i[0], str(i[0]), file_size(i[0])))
  print('\n'.join(output_lines))

# Function to get file's size and URL
def get_files_from_json(data):
  temp = []
  for i in data['Files']:
    temp.append([i['FileSize'], i['Url']])
  for i in data['Subdirectories']:
    temp.extend(get_files_from_json(i))
  return temp

# Function to show head and tail of json data
def show_head_tail(data, first = 10, last  = 10):
  print('Number of Files in JSON : {}\nFirst {} big files : \n\t{}'.format(len(data), first, '\n\t'.join(['{:15s} - {}'.format(file_size(i[0]), i[1]) for i in all_files_in_json[:first+1]])))
  print('\nLast {} files : \n\t{}'.format(last, '\n\t'.join(['{:15s} - {}'.format(file_size(i[0]), i[1]) for i in data[-last:]])))

# Function to check whether given two file's sizes are nearly equal
def is_close(a, b, n=10000):
  if a == 0:
    return True
  t = (abs(a - b)/a) * 100
  if t < 5:
    return True
  else:
    print('Difference : ', )
    return False

# Function to check whether a file completely downloaded or not
def check_progress(i):
  file_size_1 = i[0]
  file_path_1 = i[1]
  percent_encoding_list = [ ['%20', ' '], ['%21', '!'], ['%22', '"'], ['%23', '#'],
                            ['%24', '$'], ['%25', '%'], ['%26', '&'], ['%27', "'"],
                            ['%28', '('], ['%29', ')'], ['%2A', '*'], ['%2B', '+'],
                            ['%2C', ','], ['%2D', '-'], ['%2E', '.'], ['%2F', '/'],
                            ['%3A', ':'], ['%3B', ';'], ['%3C', '<'], ['%3D', '='],
                            ['%3E', '>'], ['%3F', '?'], ['%40', '@'], ['%5B', '['],
                           ['%5C', '\\'], ['%5D', ']'], ['%5E', '^'], ['%5F', '_'],
                            ['%60', '`'], ['%7B', '{'], ['%7C', '|'], ['%7D', '}'],
                            ['%7E', '~'], ['%C2%A3', '£']
                           ]

  t = i[1]
  for i in ['http://', 'https://']:
    t = t.replace(i, '')

  for i in percent_encoding_list:
    t = t.replace(i[0], i[1])
  
  file_path_2 = 'Temp/'+t

  if not os.path.exists(file_path_2):
    return 'Not Found', []
  file_size_2 = os.path.getsize(file_path_2)
  if is_close(file_size_1, file_size_2):
    return True, [file_size_1, file_path_2, file_path_1]
  else:
    return False, [file_path_1, file_size_1, file_path_2, file_size_2, '{:05.2f}%'.format(100*(file_size_2/file_size_1))]

# Function to check downloaded progress of each files
def show_progress():
  not_finished = []
  finished_downloads = []
  not_found = []
  not_found_size = 0
  for i in all_files_in_json:
    x, y = check_progress(i)
    if x == True:
      finished_downloads.append(y)
    elif x== False :
      not_finished.append(y)
    elif x == 'Not Found':
      print('404 - Not Found : ', i)
      not_found.append([i[0], i[1]])
      not_found_size += i[0]
  output_lines = []
  error_size = 0
  for i in not_finished:
    error_size += i[1]
    output_lines.append('{} - {:>10s}/{:10s} - {:80s} - {}'.format(i[4], file_size(i[3]), file_size(i[1]), i[2], i[0]))
  output_lines.sort(reverse=True)
  output_lines = ['{:03d} - {}'.format(i+1, j) for i, j in enumerate(output_lines)]
  print('\n'.join(output_lines))
  print('Not finished ({}) : {} \n'.format(len(not_finished), file_size(error_size)))
  print('404 Not Found ({}) : {}'.format(len(not_found), file_size(not_found_size)))
  return finished_downloads

# Function to show biggest n files that are downloaded
def show_biggest_in_finished(finished_downloads, n):
  output_lines = sorted(finished_downloads[:n+1], key= lambda x: x[0], reverse=True)
  output_lines = ['{:02d} - {:10s} - {:80s} - {}'.format(i+1, file_size(j[0]), j[1], j[2]) for i, j in enumerate(output_lines)]
  print('Showing biggest files in finished :\n{}'.format('\n'.join(output_lines)))

# Function to delete downloaded files
def delete_downloaded_files(created_archives_list):
  confirm = input('Please enter "Y" to delete all downloaded files, press other key to Cancel : \n')
  if confirm.lower() == 'y':
    print('Deleted Temp Folder')
    !rm -rf "Temp" >/dev/null 2>&1
    !rm -rf "test_thumb" >/dev/null 2>&1
    print('Deleted test_thumb Folder')
    !rm -rf "/content/Indexer/Scans/" >/dev/null 2>&1
    print('Deleted Scans Folder')

    !rm "running.log" >/dev/null 2>&1

    !rm "Removed.txt" >/dev/null 2>&1
    !rm "To Download.txt" >/dev/null 2>&1

    print('Deleted running.log')
    !rm "$OD_title".log
    print('Deleted {}.log'.format(OD_title))
    
    for i in created_archives_list:
      !rm "$i"
      print('Deleted ', i)
    
    !rm -rf "/content/Arc"
    print('Deleted Arc Folder')
    print('All Downloaded files are deleted :)')
  else:
    print('Deletion of Downloaded files has been Cancelled :)')

# Function to show files types info
def show_filestypes(file_types):
  file_types = [[i, file_types[i]['Count'], file_types[i]['Size']] for i in file_types]
  # print(file_types)
  file_types = sorted(file_types, key=lambda x: x[2], reverse=True)
  
  output_lines = []
  for i in file_types:
    output_lines.append('{:6s} - {:10s} - {}'.format(i[0], str(i[1]), file_size(i[2])))
  print('\n'.join(output_lines))

# Reading URL file and loading JSON data

In [ ]:
# Getting file name of URL file
temp = OD_URL
for i in ['/', '?']:
  temp = temp.replace(i, '_')
for i in os.listdir('/content/Indexer/Scans/'):
  if '.txt' in i:
    i_file    = '/content/Indexer/Scans/{}'.format(i)
    json_file = i_file.replace('.txt', '.json').replace(' - Filtered', '')

# Loading JSON
# json_file = '/content/1.json'
print('Loading Json File : {}'.format(json_file))
with open(json_file, 'r')as f1:
  json_data = json.load(f1)

# i_file = "1.txt"
print('Opening URL File : {}'.format(i_file))
if os.path.exists(i_file):
  # Opening URL file in Read mode
  with open(i_file, 'r')as f1:
    source = sorted(f1.read().splitlines())
  print('Number of record = {}'.format(len(source)))
else:
  print('URL File Not Found, Please Specify it manually')

In [ ]:
all_files_in_json = sorted(get_files_from_json(json_data['Root']), key=lambda x: x[0], reverse=True)
all_files_in_json_dict = {}
for i in all_files_in_json:
  all_files_in_json_dict[i[1]] = i[0]
file_types = find_filetypes(all_files_in_json)
show_filestypes(file_types)

### Filter URL if necessary

In [ ]:
#@title
Filter_status = True #@param {type:"boolean"}
skip_string = "/thumbs/thumbs_" #@param {type:"string"}



filtered_download_size = 0

if Filter_status :
  import re
  # Filter Input URL File
  o_file = i_file.replace('.txt', ' - Filtered.txt')
  
  # Use 0-skip or 1-accept
  skip_or_accept = 1
  if skip_or_accept == 0:
    skip_exts = ['.mp4', '.mov']
    filtered_list = sorted([i for i in source if re.search('-[0-9]+x[0-9]+', i) == None and [j for j in skip_exts if j in i.lower()] == [] and skip_string not in i])
    removed_elements_in_list = list(set(source) - set(filtered_list))
  elif skip_or_accept == 1:
    accept_exts = ['.jpg', '.png', '.jpeg']
    filtered_list = sorted([i for i in source if re.search('-[0-9]+x[0-9]+', i) == None and [j for j in accept_exts if j in i.lower()] != [] and skip_string not in i])
    removed_elements_in_list = sorted(list(set(source) - set(filtered_list)))

  print('Number of lines in input file : {}'.format(len(source)))
  print('Number of lines in Output file : {}'.format(len(filtered_list)))
  print('Number of lines in Removed : {}'.format(len(removed_elements_in_list)))

  target = [[all_files_in_json_dict[i], i] for i in filtered_list]
  target = [[i[0], quote(i[1]).replace('%3A//', '://')] for i in target]
  with open(o_file, 'w')as f1:
    f1.write('\n'.join(filtered_list))

  all_files_in_json = [i for i in all_files_in_json if i[1] not in removed_elements_in_list]
  filtered_download_size = sum([i[0] for i in all_files_in_json])
  if len(removed_elements_in_list) >11:
    print('Number of removed : {}\n{}\n{}'.format(len(removed_elements_in_list), '\n'.join(removed_elements_in_list[:11]), '\n'.join(removed_elements_in_list[-11:])))
  else:
    print('Number of removed : {}\n{}\n'.format(len(removed_elements_in_list), '\n'.join(removed_elements_in_list[:11])))
  print('Avoided Unneccessary Download Size : {} and Files Count : {}'.format(file_size(json_data['TotalFileSizeEstimated'] - filtered_download_size), len(removed_elements_in_list)))
  
  with open('Removed.txt', 'w') as f1:
    f1.write('\n'.join(removed_elements_in_list))
else:
  filtered_download_size = json_data['TotalFileSizeEstimated']
  target = [[all_files_in_json_dict[i], i] for i in source]
  target = [[i[0], quote(i[1]).replace('%3A//', '://')] for i in target]

with open('To Download.txt', 'w')as f1:
  f1.write('\n'.join([i[1] for i in target]))

print('To Download Size : {}'.format(file_size(filtered_download_size)))
show_head_tail(all_files_in_json, 10, 10)

# Calling Downloader function

In [ ]:
# Calling Download function
mb_size = [2**20]
total_mb_size = [int(filtered_download_size/mb_size[0])]
downloaded_size = [0]
tqdm_bar = tqdm(total=len(target), desc='Count', unit=' Count')
tqdm_bar_size = tqdm(total=filtered_download_size, unit=' Bytes')
start_thread(target, 10)
tqdm_bar.close()
tqdm_bar_size.close()
# print('All Downloads are Started.\nFiles with fast speed will be downloaded instanly.\nSlow Speed Downloads tends to take time.\nRun folder_size function to check progress of downloads')

    The Downloader will initiate wget for each file.

    If a file size is larger than 1GB than it may took a while to get downloaded in the background.

    Run below cell to calculate current downloaded size

## Calculate Downloaded size

In [ ]:
#while True:
# Calculating Folder Size
i = get_folder_size('Temp')
downloaded_size = i[1]
total_download_size = filtered_download_size
percent_downloaded = (downloaded_size/total_download_size) * 100
print('\r{:.2f} % Finished - ({}/{}) - {} to download'.format(percent_downloaded, file_size(downloaded_size), file_size(total_download_size), file_size(total_download_size-downloaded_size)), end='')
if percent_downloaded > 95 :
  print('\nDownloading Finished')
  #break
#time.sleep(1)

In [ ]:
# total_files = json_data['TotalFiles']
# total_size = json_data['TotalFileSizeEstimated']
# speed_test = '{} per Second'.format(file_size(int(json_data['SpeedtestResult']['DownloadedBytes']/(json_data['SpeedtestResult']['ElapsedMilliseconds']/1000))))

finished_downloads = show_progress()

In [ ]:
show_biggest_in_finished(finished_downloads, 10)

Currently Running Process

In [ ]:
!ps aux | tee running.log

In [ ]:
with open('running.log', 'r')as f1:
  wget_processes = len([i for i in f1.read().splitlines() if 'wget' in i])
print('{} instances of Wget are running'.format(wget_processes))

In [ ]:
show_folder_size('Temp')

# Archiver

In [ ]:
# Copy Scans folder to Temp folder
index_files_prefix = unquote(OD_URL.replace('/', '_'))
index_files = ['/content/Indexer/Scans/'+i for i in os.listdir('/content/Indexer/Scans/') if index_files_prefix in i]
scan_path = '/content/Temp/Scans'
if not os.path.exists(scan_path):
  os.makedirs(scan_path)
for i in index_files:
  !rsync -ah --progress "$i" "$scan_path"

In [ ]:
# Creating name for Archive
i_folder = "Temp"
o_arc_file = OD_title
# o_arc_file = "PB"
archive_folder = 'Arc'
if not os.path.exists(archive_folder):
  os.makedirs(archive_folder)
o_arc_file = '{}/{} - {}.7z'.format(archive_folder, dati.now(timezone('Asia/Kolkata')).strftime('%Y-%m-%d_%H-%M-%S'), o_arc_file)
o_arc_file

In [ ]:
## Compressing folder with 7Zip
# a    - Create Archive
# m0   - Specify Compression
# Copy - No Compression
# v    - Split archive with size specified
# 1g   - Split archive with each 1GB

i = get_folder_size('Temp')
downloaded_size = i[1]
one_gb_size = 2**30

# import getpass
# star_pas = getpass.getpass()
star_pas = "1234567890"
!7z a -m0=Copy "$o_arc_file" "$i_folder" -v500m -mhe -p"$star_pas"

In [ ]:
# Test archived files
!7z t "$o_arc_file".001 -p$star_pas

# Copier

In [ ]:
# Copy created archive to Google Drive
created_archives_list = []
copy_to = "/content/drive/My Drive/My Files/Temp"

if not os.path.exists(copy_to):
  os.makedirs(copy_to)

copying_time = time.time()
# List files in current directory
for i in sorted(os.listdir(archive_folder)):
  # Check if current file is a archive
  if os.path.split(o_arc_file)[1] in i :
    created_archives_list.append(os.path.join(archive_folder, i))
  elif "Thumbnails.7z" in i :
    created_archives_list.append(os.path.join(archive_folder, i))

print(len(created_archives_list))
created_archives_list

In [ ]:
# Copying each files to Destination
copy_archive_list = created_archives_list
copy_archive_list

In [ ]:
for i, j in enumerate(copy_archive_list):
  print(j)
  print('\n\n{}/{} : ({}) {}'.format(i+1, len(copy_archive_list), file_size(os.path.getsize(j)), j))
  temp_cp_time = time.time()
  !rsync -ah --progress "$j" "$copy_to"
  temp_cp_time = time.time() - temp_cp_time
  print('Time Took to Copy this file : {}'.format(time.strftime('%H-%M-%S', time.gmtime(temp_cp_time))))

copying_time = time.time() - copying_time
print('\nTotal time took to copy files : {}'.format(time.strftime('%H-%M-%S', time.gmtime(copying_time))))

In [ ]:
show_folder_size(copy_to)

# Delete Downloaded files

In [ ]:
# Delete Downloaded files
delete_downloaded_files(created_archives_list)

# Gofile.io Uploading

If you want to use Gofile.io, Then change enabled to True

In [ ]:
enabled = False

## Defining Functions for Gofile.io API

In [ ]:
def get_server():
  print('Getting Fastest Gofile.io server.',end='')
  server_checking_url = 'https://apiv2.gofile.io/getServer'
  log_file = '1.log'
  !curl $server_checking_url >$log_file 2>/dev/null
  print('.', end='')
  fast_server_info = !cat $log_file
  fast_server_info = json.loads(fast_server_info[0])
  if fast_server_info['status'] == 'error':
    print('Cannot get Server URL')
    return 'Error'

  upload_url = 'https://{}.gofile.io/uploadFile'.format(fast_server_info['data']['server'])
  print('.{}\n'.format(upload_url))
  return upload_url


def upload_file(file_name, email):
  upload_url = get_server()

  if upload_url != 'Error':
    if not os.path.exists(file_name):
      print('File to upload not found : {}'.format(file_name))
      return {}
    else:
      log_file = "1.log"
      !curl -F email=$email -F file=@"$file_name" $upload_url >$log_file
      response = !cat $log_file

      if response == []:
        print('No Response from server')
        return {}
      response = json.loads(response[0])
      
      if response['status'] == 'error':
        print('Error uploading file')
        return {}
      response = response['data']
      response['url_of_file'] = 'https://gofile.io/d/{}'.format(response['code'])
      return response

def show_upload_file_info(response):
  if response != {}:
    print('\n\nFile name   : {}'.format(response['file']['name']))
    print('Admic code to file : {}'.format(response['adminCode']))
    print('URL to file : {}\n\n'.format(response['url_of_file']))


def acc_info(token):
  print('Getting Account Info')
  account_info_url = 'https://apiv2.gofile.io/getAccountInfo?token={}'.format(token)
  log_file = "1.log"
  !curl $account_info_url >$log_file 2>/dev/null
  res = !cat $log_file
  res = json.loads(res[0])
  if res['status'] == 'error':
    print('Invalid Token Given')
    return {}
  
  res = res['data']
  return res

def show_acc_info(result):
  if result != {}:
    print('\nAccount Info : \n\tEmail   : {}\n\tAccount : {}\n\tFiles Count : {}\n\tFiles Size  : {}'.format(result['email'], result['account'], result['filesCount'], file_size(result['filesSize'])))


def user_uploads_info(token):
  print('Getting Uploads Info')
  files_info_url   = 'https://apiv2.gofile.io/getUploadsList?token={}'.format(token)
  log_file = '1.log'
  !curl $files_info_url >$log_file 2>/dev/null
  res = !cat $log_file
  r = json.loads(res[0])
  if r['status'] == 'error':
    print('Invalid Token Given')
    return {}

  r = r['data']
  for index_i, i in enumerate(r):
    r[i]['File_URL'] = 'https://gofile.io/d/{}'.format(r[i]['code'])
  return r

def show_user_uploads_info(r):
  if r != {}:
    uploads_count = len(r)
    print('Uploads Info :\nNumber of Uploads : {}'.format(uploads_count))

    for index_i, i in enumerate(r):
      link = r[i]['File_URL']
      files_count = len(r[i]['files'])
      temp_output = []

      for index_j, j in enumerate(r[i]['files']):
        name = r[i]['files'][j]['name']
        size = file_size(r[i]['files'][j]['size'])
        
        try:
          md5  = r[i]['files'][j]['md5']
        except:
          md5  = 'Unknown'
        temp_output.append('({}/{}) : {}\n\t\t\tMD5  : {}\n\t\t\tSize : {}'.format(index_j+1, files_count, name, md5, size))

      try:
        views = r[i]['views']
      except:
        views = 'Unknown'
      
      downloads = r[i]['downloaded']
      total_size = file_size(r[i]['totalSize'])
      uploaded_time = dati.utcfromtimestamp(int(r[i]['uploadTime'])).strftime('%Y-%m-%d %H:%M:%S')
      expire_time   = dati.utcfromtimestamp(int(r[i]['removalDate'])).strftime('%Y-%m-%d %H:%M:%S')
      admin_code    = r[i]['adminCode']

      t = '\t({}/{}) : {}\n\t\tFiles Count : {}\n\t\tViews      : {}\n\t\tDownloads  : {}\n\t\tTotal Size : {}\n\t\tUploaded Time : {}\n\t\tExpiry Date   : {}\n\t\tAdmin Code    : {}\n\t\tFiles : \n\t\t\t{}'
      print(t.format(index_i+1, uploads_count, link, files_count, views, downloads, total_size, uploaded_time, expire_time, admin_code, '\n'.join(temp_output)))


## Upload File

In [ ]:
# Use your email address if you have account in Gofile.io
file_name = '/content/sample_data/california_housing_train.csv'
email = 'yourgmailid@gmail.com'

if enabled == True:
  result = upload_file(file_name, email)
  show_upload_file_info(result)
else:
  print('Gofile.io Operations not enabled\nTo enable this, Change value of variable named "enabled" to True ')

## Batch Upload Single Files

In [ ]:
email = '123@gmail.com'
files_to_upload = [i for i in os.listdir('.') if o_arc_file in i]
files_to_upload

In [ ]:
# Upload Multiple Files
uploads_details = []
if enabled == True:
  for i in files_to_upload:
    result = upload_file(i, email)
    show_upload_file_info(result)
else:
  print('Gofile.io Operations not enabled\nTo enable this, Change value of variable named "enabled" to True ')

## Get Account Info

In [ ]:
# Pass your Gofile.io Account's Token
token = 'gdfgdfgdgf241ce713d13bfccbfd77c0362357c'

if enabled == True:
  result = acc_info(token)
  show_acc_info(result)
else:
  print('Gofile.io Operations not enabled\nTo enable this, Change value of variable named "enabled" to True ')

## Uploaded Files Info


In [ ]:
# Pass your Gofile.io Account's Token
token = 'c8b22a44a8c7ab3a6241ce713d13bfccbdfgdfgdfgdfg357c'

if enabled == True:
  result = user_uploads_info(token)
  show_user_uploads_info(result)
else:
  print('Gofile.io Operations not enabled\nTo enable this, Change value of variable named "enabled" to True ')

# Testing Downloads

## Video thumbnail Generator function

In [ ]:
# Download Aerial.ttf Font
!wget "https://drive.google.com/uc?id=1jZDDngJNKvnDC6vn6rzUobN0SDuifiDK&export=download" -O "Aerial.ttf"
# Refer : https://stackoverflow.com/questions/27568254/how-to-extract-1-screenshot-for-a-video-with-ffmpeg-at-a-given-time
import subprocess
import os
import time
import math

from tqdm import tqdm
from multiprocessing.dummy import Pool

from datetime import datetime as dati
from pytz import timezone

from tqdm.notebook import tqdm
from google.colab import output as colab_output
from IPython.display import Image as colab_image

from PIL import Image
from PIL import ImageFile
from PIL import ImageFont
from PIL import ImageDraw
ImageFile.LOAD_TRUNCATED_IMAGES = True

def ffmpeg_get_screenshot(i):
  with open(os.devnull, 'wb') as devnull:
    subprocess.check_call(['ffmpeg', '-ss', i[0], '-i', i[1], '-y', '-vframes', '1', i[2]], stdout=devnull, stderr=subprocess.STDOUT)

def start_collage(list_of_images, label_list, video_thumb_folder, video_name):
  n = len(list_of_images)
  matrix_size = 7
  matrix_total_size = matrix_size * matrix_size
  number_of_collages = n / (matrix_total_size)
  number_of_collages = int(number_of_collages)+1 if number_of_collages > int(number_of_collages) else number_of_collages
  
  black_webp = 'Temp_Blank.webp'
  bl_img = Image.new('RGB', (1000, 1000))
  bl_img.save(black_webp, "WEBP")

  blank_img_to_add = (matrix_total_size*number_of_collages) - n
  for i in range(blank_img_to_add):
    list_of_images.append(black_webp)
    label_list.append('')
  
  for i in range(number_of_collages):
    output_collage = "{}/_{} - {}.webp".format(video_thumb_folder, video_name.replace('.', '_'), i+1)
    create_collage(list_of_images[matrix_total_size*i:matrix_total_size*(i+1)], output_collage, img_type='WEBP', width=2000, height=2000, cols=7, rows=7, label_list=label_list[matrix_total_size*i:matrix_total_size*(i+1)])
  os.remove(black_webp)

def create_collage(listofimages, o_file, img_type='WEBP', width=2000, height=2000,cols=5,rows=4, label_list=[]):
  if cols == 0:
    return
  max_cells = 100
  thumbnail_width = width//cols
  thumbnail_height = height//rows
  size = thumbnail_width, thumbnail_height
  new_im = Image.new('RGB', (width, height), (100, 100, 100))
  
  draw = ImageDraw.Draw(new_im)
  font = ImageFont.truetype('Aerial.ttf', 20)
  ims = []
  text_coord = []
  for p in listofimages:
    try:
      im = Image.open(p)
    except:
      im = Image.new('RGB', (500,500))
    im.thumbnail(size)
    ims.append(im)
  i = x = y = 0
  for row in range(rows):
    for col in range(cols):
      text_coord.append([x + thumbnail_width*.3, y + thumbnail_height*.7])
      new_im.paste(ims[i], (x, y))      
      i += 1
      x += thumbnail_width
    y += thumbnail_height
    x = 0
  for index_i, i in enumerate(text_coord):
    draw.text((i[0], i[1]), label_list[index_i],(255,255,255),font=font)
  new_im.save(o_file,img_type)

def video_to_img(video_folder, video_name, video_thumb_folder, for_seconds=60):
  video_path = "{}/{}".format(video_folder, video_name)
  result = subprocess.run(['ffprobe', '-v', 'error', '-show_entries', 'format=duration', '-of', 'default=noprint_wrappers=1:nokey=1', video_path], stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
  video_duration = float(result.stdout)
  no_of_imgs = int(video_duration/for_seconds)

  if not os.path.exists(video_thumb_folder):
    os.makedirs(video_thumb_folder)
  
  print('{:>5s} Seconds - {:>6s} Parts - Path : {}\n'.format(str(int(video_duration)), str(no_of_imgs), video_path))
  zeros_size = len(str(no_of_imgs))
  list_of_images = []
  label_list = []
  ffmpeg_commands = []

  for i in range(1, no_of_imgs+1 ):
    time_pointer =  time.strftime('%H:%M:%S', time.gmtime(i*for_seconds))
    label_list.append(time_pointer)
    out_img_path = "{}/{}.jpg".format(video_thumb_folder, time_pointer.replace(':', '_'))
    list_of_images.append(out_img_path)
    ffmpeg_commands.append([time_pointer, video_path, out_img_path])
  
  with Pool(10) as p:
    r = list(tqdm(p.imap(ffmpeg_get_screenshot, ffmpeg_commands), total=len(ffmpeg_commands)))
  
  start_collage(list_of_images, label_list, video_thumb_folder, video_name)
  print('\nPhoto Collage Created')

In [ ]:
vi_in_temp = []
for path, dirs, files in os.walk('Temp'):
  for file in files:
    if '.mp4' in file:
      vi_in_temp.append([path, file])
print('Number of Files : ', len(vi_in_temp))
print('\n'.join(str(i) for i in vi_in_temp[:20]))

In [ ]:
tqdm_bar = tqdm(total=len(vi_in_temp))

In [ ]:
vi_thumbnail_folder = '/content/Vi_Thumbnails'
for i in vi_in_temp:
  video_folder = i[0]
  video_name = i[1]
  video_thumb_folder = "{}/{}-Thumbs".formatvi_thumbnail_folder, (os.path.splitext(video_name)[0])
  if not os.path.exists(video_thumb_folder):
    video_to_img(video_folder, video_name, video_thumb_folder, 10)
  tqdm_bar.update(1)
  colab_output.clear()
tqdm_bar.close()

In [ ]:
show_folder_size(get_folder_size(vi_thumbnail_folder))

In [ ]:
## Need To Create Collage

## GIF Creation

In [ ]:
from IPython.display import display
from ipywidgets import Dropdown
from PIL import Image

def create_small_gif(temp_folder_1, out_gif, gif_size):
  filenames = sorted(['{}/{}'.format(temp_folder_1, i) for i in os.listdir(temp_folder_1)])
  img = Image.new('RGB', gif_size)
  imgs = [Image.open(f) for f in filenames if os.path.splitext(f)[1].lower() in ['.jpg', '.png', '.jpeg', '.gif' ]]
  for i in imgs:
    i.thumbnail(gif_size)
  img.save(fp=out_gif, format='GIF', append_images=imgs, save_all=True, duration=200, loop=0)

def folder_to_gifs(folder, out_folder, resolution):
  if not os.path.exists(out_folder):
    os.makedirs(out_folder)
  temp_folder_gifs = ['{}/{}'.format(folder, i) for i in sorted(os.listdir(folder))]
  print(len(temp_folder_gifs))
  print(temp_folder_gifs[0])
  

  tqdm_bar = tqdm(total=len(temp_folder_gifs))
  for i in temp_folder_gifs:
    create_small_gif(i, "{}/{}.gif".format(out_folder, os.path.split(i)[-1]), resolution)
    tqdm_bar.update(1)
  tqdm_bar.close()
  #############################################################################
  ##########################################################################
  ################################################################
  ######################################################

def dropdown_eventhandler(change):
  colab_output.clear()
  display(dropdown)
  
  def dropdown_img_eventhandler(change):
    colab_output.clear()
    display(dropdown)
    display(dropdown_img)

    current_img = change.new
    print(current_img)
    current_img = current_img[current_img.find('/'):]
    print(current_img)

    im = Image.open(current_img)
    im.thumbnail((100, 100))
    im.save('temp.jpeg', 'JPEG')
    display(colab_image('temp.jpeg'))

  current_folder = change.new
  current_folder = "/content/Vi_Thumbnails/{}".format(current_folder[current_folder.find('. ')+2:])
  sorted_folders = [ '{}/{}'.format(current_folder, i) for i in sorted(os.listdir(current_folder))]
  option_list = ['{}. ({}) {}'.format(index_i+1, file_size(os.path.getsize(i)), i) for index_i, i in enumerate(sorted_folders)]
  dropdown_img = Dropdown(description="Imgs : ", options=option_list)
  dropdown_img.observe(dropdown_img_eventhandler, names='value')
  display(dropdown_img)


In [ ]:
gif_in_folder = "/content/Vi_Thumbnails"
gif_out_folder = "Medium_gifs"
folder_to_gifs(gif_in_folder, gif_out_folder, (500, 500))

In [ ]:
colab_image(open('image.gif','rb').read())

In [ ]:
out_archive_file = '{} - {}.7z'.format(dati.now(timezone('Asia/Kolkata')).strftime('%Y-%m-%d_%H-%M-%S'), os.path.split(gif_out_folder)[-1])
out_archive_file

In [ ]:
!7z a -m0=Copy "$out_archive_file" "$gif_out_folder" -mhe -p1234567890 -v300m

In [ ]:
!7z t "$out_archive_file".001 -p1234567890

In [ ]:
archives_list = []
for i in sorted(os.listdir()):
  if out_archive_file in i:
    archives_list.append(i)
archives_list

In [ ]:
for i in archives_list:
  !rsync -ah --progress "$i" "/content/drive/My Drive/"

In [ ]:
sorted_folders = sorted(os.listdir("/content/Vi_Thumbnails"))
option_list = ['{}. {}'.format(index_i+1, i) for index_i, i in enumerate(sorted_folders)]
dropdown = Dropdown(description="Vi_Th Folders : ", options=option_list)
dropdown.observe(dropdown_eventhandler, names='value')
display(dropdown)

## Download GD Files

In [ ]:
!pip install googledrivedownloader

In [ ]:
drive_link = "https://drive.google.com/u/0/uc?id=1234q4mopxvZPwSPN4_s5EqY8sq3M_H&export=download"
drive_link_id = "1234q4mopxvZPwSPN4_s5EqY8sq3M_H"
download_file_name = "Data/1.zip" #Should Contain folder naem
to_unzip = False

from google_drive_downloader import GoogleDriveDownloader as gdd
gdd.download_file_from_google_drive(file_id=drive_link_id, dest_path=download_file_name, unzip=to_unzip)

In [ ]:
!rsync -ah --progress "1.zip" "/content/drive/My Drive"

## Compress Video with FFMPEG YIFY-Settings

In [ ]:
download_file_name = "/content/Tom and Jerry Tales s01 ep09.avi"
converted_file_name = "/content/Tom and Jerry Tales s01 ep09 - Converted.mkv"

In [ ]:
!ffprobe "$download_file_name" -show_entries format=nb_streams -v 0 -of compact=p=0:nk=1

In [ ]:
!ffmpeg -i "$download_file_name" -c:v libx264 -crf 27 -x264-params cabac=1:ref=5:analyse=0x133:me=umh:subme=9:chroma-me=1:deadzone-inter=21:deadzone-intra=11:b-adapt=2:rc-lookahead=60:vbv-maxrate=10000:vbv-bufsize=10000:qpmax=69:bframes=5:b-adapt=2:direct=auto:crf-max=51:weightp=2:merange=24:chroma-qp-offset=-1:sync-lookahead=2:psy-rd=1.00,0.15:trellis=2:min-keyint=23:partitions=all -c:a aac -ar 44100 -b:a 128k -map 0 "$converted_file_name"

In [ ]:
from IPython.display import FileLink, FileLinks
display(FileLink("/content/sample_data/california_housing_test.csv"))

In [ ]:
FileLinks('/content/sample_data')

## Download and Compress

In [ ]:
url     = "https://www.zip"
file_to = "1.zip"
password = "1234567890"

In [ ]:
import os

ext_folder = os.path.splitext(file_to)[0]
print(ext_folder)
arc_file   = os.path.splitext(file_to)[0] + '.7z'
print(arc_file)

!wget -c $url -O "$file_to"

In [ ]:
!7z x "$file_to" -O"$ext_folder"

In [ ]:
!7z a -m0=lzma2 -mx=9 "$arc_file" "$ext_folder" -mhe -p"$password"

In [ ]:
!7z t "$arc_file" -p"$password"

In [ ]:
# Mounting Google Drive to this Virtual Machine
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!rsync -ah --progress "$arc_file" "/content/drive/My Drive/$arc_file"

## PDF Extractor

### Premilinaries

In [ ]:
!pip install PyPDF2
!pip install textract
!pip install rake_nltk

In [ ]:
import pandas as pd
import numpy as np
import PyPDF2
import textract
import re

### Reading Text
> Converted PDF file to txt format for better pre-preprocessing

In [ ]:
filename ='/content/Deep Learning 2020/pytorch-Deep-Learning-master/slides/04 - RNN.pdf'

pdfFileObj = open(filename,'rb')               #open allows you to read the file
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)   #The pdfReader variable is a readable object that will be parsed
num_pages = pdfReader.numPages                 #discerning the number of pages will allow us to parse through all the pages


count = 0
text = ""
                                                            
while count < num_pages:                       #The while loop will read each page
  pageObj = pdfReader.getPage(count)
  count +=1
  text += pageObj.extractText()
    
#Below if statement exists to check if the above library returned #words. It's done because PyPDF2 cannot read scanned files.

if text != "":
  text = text
    
#If the above returns as False, we run the OCR library textract to #convert scanned/image based PDF files into text

else:
  text = textract.process('http://bit.ly/epo_keyword_extraction_document', method='tesseract', language='eng')

  # Now we have a text variable which contains all the text derived from our PDF file.

In [ ]:
text = text.encode('ascii','ignore').lower().decode("utf-8")  #Lowercasing each word
text

### Extracting Keywords

In [ ]:
keywords = re.findall(r'[a-zA-Z]\w+',text)
len(keywords)                               #Total keywords in document

In [ ]:
df = pd.DataFrame(list(set(keywords)),columns=['keywords'])  #Dataframe with unique keywords to avoid repetition in rows

### Calculating Weightage
> In information retrieval, tf–idf or TFIDF, short for term frequency–inverse document frequency, is a numerical statistic that is intended to reflect how important a word is to a document in a collection or corpus.It is often used as a weighting factor in searches of information retrieval, text mining, and user modeling.


> **TF: Term Frequency**, which measures how frequently a term occurs in a document. Since every document is different in length, it is possible that a term would appear much more times in long documents than shorter ones. Thus, the term frequency is often divided by the document length (aka. the total number of terms in the document) as a way of normalization:


**TF(t) = (Number of times term t appears in a document) / (Total number of terms in the document).**

> **IDF: Inverse Document Frequency**, which measures how important a term is. While computing TF, all terms are considered equally important. However it is known that certain terms, such as "is", "of", and "that", may appear a lot of times but have little importance. Thus we need to weigh down the frequent terms while scale up the rare ones, by computing the following:

**IDF(t) = log_e(Total number of documents / Number of documents with term t in it).**

In [ ]:
def weightage(word,text,number_of_documents=1):
  word_list = re.findall(word,text)
  number_of_times_word_appeared =len(word_list)
  tf = number_of_times_word_appeared/float(len(text))
  idf = np.log((number_of_documents)/float(number_of_times_word_appeared))
  tf_idf = tf*idf
  return number_of_times_word_appeared,tf,idf ,tf_idf

In [ ]:
df['number_of_times_word_appeared'] = df['keywords'].apply(lambda x: weightage(x,text)[0])
df['tf'] = df['keywords'].apply(lambda x: weightage(x,text)[1])
df['idf'] = df['keywords'].apply(lambda x: weightage(x,text)[2])
df['tf_idf'] = df['keywords'].apply(lambda x: weightage(x,text)[3])

In [ ]:
df = df.sort_values('tf_idf',ascending=True)
df.to_csv('Keywords.csv')
df.head(25)

### Second Method - Using Gensim library


In [ ]:
from gensim.summarization import keywords
import warnings
warnings.filterwarnings("ignore")

In [ ]:
values = keywords(text=text,split='\n',scores=True)

In [ ]:
data = pd.DataFrame(values,columns=['keyword','score'])
data = data.sort_values('score',ascending=False)
data.head(10)

### Third Approach - Using RAKE (Rapid Automatic Keyword Extraction)

In [ ]:
from rake_nltk import Rake

In [ ]:
r = Rake()
r.extract_keywords_from_text(text)

In [ ]:
phrases = r.get_ranked_phrases_with_scores()

In [ ]:
table = pd.DataFrame(phrases,columns=['score','Phrase'])
table = table.sort_values('score',ascending=False)
table.head(10)

## IMG Thumbnail generator (Incomplete)

In [ ]:
def thumb_gen(i):
  size = 128, 128
  outfile = i[1]
  infile = i[0]
  n = 0
  while True:
    n += 1
    if os.path.exists(outfile):
      temp = os.path.splitext(i[1])
      outfile = '{} - {}.{}'.format(temp[0], n, temp[1])
    else:
      break
  print(infile, outfile)
  if infile != outfile:
    try:
      im = Image.open(infile)
      im.thumbnail(size)
      im.save(outfile, "JPEG")
    except IOError:
      print('Error : ',i)

def create_thumbnails():
  to_thumb_list = []
  thumbnails_ext = ['.jpg', '.jpeg', '.png']
  src_to_thumb = '/content/Temp/'
  thumb_out = '/content/test_thumb/'
  if not os.path.exists(thumb_out):
    os.makedirs(thumb_out)
  for path, dirs, files in os.walk(src_to_thumb):
    for file in files:
      if os.path.splitext(file)[1] in thumbnails_ext:
        test_thumb_folder = path.replace(src_to_thumb, thumb_out)
        if not os.path.exists(test_thumb_folder):
          os.makedirs(test_thumb_folder)
        to_thumb_list.append([os.path.join(path, file), test_thumb_folder+'/'+file])
  
  print('First record : \n', to_thumb_list[0])
  input('Sample : ')
  print('Number of Imgs to Thumbnail : {}'.format(len(to_thumb_list)))
  thumb_out = 'test_thumb/'
  !rm -rf "$thumb_out"
  if not os.path.exists(thumb_out):
    os.makedirs(thumb_out)
  with Pool_mp(1) as p:
    r = list(tqdm(p.imap(thumb_gen, to_thumb_list[:10]), total=len(to_thumb_list[:10])))
  print('Number of thumbnails  : {}\n\n'.format(len(os.listdir('test_thumb'))))
  o_arc_file = '{} - {} Thumbnails.7z'.format(dati.now(timezone('Asia/Kolkata')).strftime('%Y-%m-%d_%H-%M-%S'), OD_title)
  !7z a -m0=Copy "$o_arc_file" "$thumb_out"

In [ ]:
temp = "/content/Temp/them/images"
a = '{}/{}'.format(temp, os.listdir(temp)[0])
b = a.replace('/content/Temp', '/content/test_thumb')
os.makedirs(os.path.split(temp)[0])
# print(a, b)

im = Image.open(a)
im.thumbnail((128, 128))
im.save(b, "JPEG")

In [ ]:
enabled_thumb = True
if enabled_thumb == True:
  create_thumbnails()

## Video To IMG 

In [ ]:
# Install FFMPEG Python Package
print('Installing FFMPEG', end='')
!pip install ffmpeg-python >/dev/null 2>&1

# Download Aerial.ttf Font
!wget "https://drive.google.com/uc?id=1jZDDngJNKvnDC6vn6rzUobN0SDuifiDK&export=download" -O "Aerial.ttf" >/dev/null 2>&1

# Refer : https://stackoverflow.com/questions/27568254/how-to-extract-1-screenshot-for-a-video-with-ffmpeg-at-a-given-time
import subprocess
import os
import time
import math

import ffmpeg
from tqdm.notebook import tqdm
from multiprocessing.dummy import Pool

from datetime import datetime as dati
from pytz import timezone

from PIL import Image
from PIL import ImageFile
from PIL import ImageFont
from PIL import ImageDraw

ImageFile.LOAD_TRUNCATED_IMAGES = True

def file_size(i):
  n, p, d = 0, 1024, ['Bytes', 'KB', 'MB', 'GB', 'TB']
  while i > p:
    i /= p
    n += 1
  return '{:.2f} {}'.format(i, d[n])

def dati_now():
  return dati.now(timezone('Asia/Kolkata')).strftime("%Y-%m-%d_%H-%M-%S")

def ffmpeg_get_screenshot(i):
  with open(os.devnull, 'wb') as devnull:
    subprocess.check_call(['ffmpeg', '-ss', i[0], '-i', i[1], '-y', '-vframes', '1', i[2]], stdout=devnull, stderr=subprocess.STDOUT)

def start_collage(list_of_images, label_list, video_thumb_folder, video_name, vi_de):
  
  n = len(list_of_images)
  number_of_columns = 7
  number_of_rows    = 7
  matrix_total_size = number_of_rows * number_of_columns
  number_of_collages = n / (matrix_total_size)
  number_of_collages = int(number_of_collages)+1 if number_of_collages > int(number_of_collages) else int(number_of_collages)
  collage_img_type = 'JPEG'

  black_webp = 'Temp_Blank.{}'.format(collage_img_type)
  bl_img = Image.new('RGB', (1000, 1000))
  bl_img.save(black_webp, collage_img_type)

  blank_img_to_add = (matrix_total_size*number_of_collages) - n
  for i in range(blank_img_to_add):
    list_of_images.append(black_webp)
    label_list.append('')
  
  for i in range(number_of_collages):
    output_collage = "{}/C_{} - {}.{}".format(video_thumb_folder, video_name.replace('.', '_'), i+1, collage_img_type.lower())
    create_collage_image(list_of_images[matrix_total_size*i:matrix_total_size*(i+1)], output_collage, vi_de, img_type=collage_img_type, width=2000, height=2000, cols=number_of_columns, rows=number_of_rows, label_list=label_list[matrix_total_size*i:matrix_total_size*(i+1)], frame_index=i)
  os.remove(black_webp)

def create_collage_image(listofimages, o_file, vi_de, img_type='WEBP', width=2000, height=2000,cols=5,rows=4, label_list=[], frame_index=1):
  
  if cols == 0:
    return
  new_im = Image.new('RGB', (width, height), (20, 20, 20))
  draw = ImageDraw.Draw(new_im)
  font = ImageFont.truetype('Aerial.ttf', 20)
  
  # Details Pane
  details_pane_percentage = 0.1
  details_pane_height_end = int(height * details_pane_percentage)
  
  de_file_name  = vi_de['format']['filename']
  de_file_size  = file_size(int(vi_de['format']['size']))
  de_duration   = time.strftime('%H H: %M M: %S S', time.gmtime(float(vi_de['format']['duration'])))
  de_framerate  = vi_de['streams'][0]['r_frame_rate'].split('/')
  de_framerate  = '{:.2f} frames/sec'.format(int(de_framerate[0])/int(de_framerate[1]))
  de_video_type = '{} ({})'.format(vi_de['format']['format_long_name'], vi_de['streams'][0]['codec_long_name'])
  de_audio_type = '{} ({}), {}Hz'.format(vi_de['streams'][1]['codec_name'], vi_de['streams'][1]['channel_layout'], vi_de['streams'][1]['sample_rate'])
  
  de_subtitle   = "No Subtitle"
  for i in vi_de['streams']:
    if i["codec_type"] == "subtitle":
      de_subtitle   = i["codec_long_name"]
      break
  
  details = [
              "Video Name : {} ({})".format(de_file_name, de_file_size),
              "Timestamp   : {}, Collage-{}".format(dati_now(), frame_index),
              "Duration       : {}".format(de_duration),
              "Resolution    : {}x{} / {}".format(vi_de['streams'][0]['width'], vi_de['streams'][0]['height'], de_framerate),
              "Video Type   : {}".format(de_video_type),
              "Audio Type   : {}".format(de_audio_type),
              "Subtitles      : {}".format(de_subtitle)
            ]
  
  details_pane = Image.new('RGB', (width, details_pane_height_end), (70, 70, 70))
  details_draw = ImageDraw.Draw(details_pane)
  
  details_line_height = int(details_pane_height_end/(len(details)+2))
  x = int(width*0.07)
  y = details_line_height
  
  for i in details:
    details_draw.text((x, y), i,(255,255,255),font=font)
    y += details_line_height
  new_im.paste(details_pane, (0, 0))
  
  # Pictures Pane
  collage_width  = int(width  * (0.9))
  collage_height = int(height * (1-details_pane_percentage))
  thumbnail_width = collage_width//cols
  thumbnail_height = collage_height//rows
  size = thumbnail_width, thumbnail_height
  ims = []
  text_coord = []
  
  for p in listofimages:
    try:
      im = Image.open(p)
    except:
      im = Image.new('RGB', (500,500))
    im.thumbnail(size)
    ims.append(im)
    
  i = 0
  x = int(thumbnail_width*0.1)
  y = int(details_pane_height_end * 1.1)
  for row in range(rows):
    for col in range(cols):
      text_coord.append([x + thumbnail_width*.2, y + thumbnail_height*.55])
      new_im.paste(ims[i], (x, y))
      i += 1
      x += int(thumbnail_width * 1.1)
    y += int(thumbnail_height * 0.9)
    x = int(thumbnail_width*0.1)
    
  for index_i, i in enumerate(text_coord):
    draw.text((i[0], i[1]), label_list[index_i],(255,255,255),font=font)

  new_im.save(o_file,img_type)

def video_to_img(video_path, video_thumb_folder, for_seconds=60):
  
  video_thumb_folder = "{}/{} - {}_Thumbs".format(video_thumb_folder, dati_now(), os.path.splitext(os.path.split(video_path)[1])[0])
  video_name = os.path.split(video_path)[1]
  
  if not os.path.exists(video_thumb_folder):
    os.makedirs(video_thumb_folder)
    
  if not os.path.exists(video_path):
    print('File does not Exists :', video_path)
    return
  
  vi_de = ffmpeg.probe(video_path)    
  video_duration = float(vi_de['format']['duration'])
  no_of_imgs = int(video_duration/for_seconds)
  
  print('{:>5s} Seconds - {:>6s} Parts - Path : {}\n'.format(str(int(video_duration)), str(no_of_imgs), video_path))
  zeros_size = len(str(no_of_imgs))
  list_of_images = []
  label_list = []
  ffmpeg_commands = []
  
  time_pointer_count = 1
  for i in range(1, no_of_imgs+1 ):
    time_pointer =  time.strftime('%H:%M:%S', time.gmtime(i*for_seconds))
    label_list.append('({}) {}'.format(time_pointer_count, time_pointer))
    time_pointer_count += 1
    
    out_img_path = "{}/F_{}.jpg".format(video_thumb_folder, time_pointer.replace(':', '_'))
    list_of_images.append(out_img_path)
    ffmpeg_commands.append([time_pointer, video_path, out_img_path])
  
  with Pool(10) as p:
    r = list(tqdm(p.imap(ffmpeg_get_screenshot, ffmpeg_commands), total=len(ffmpeg_commands)))
  
  start_collage(list_of_images, label_list, video_thumb_folder, video_name, vi_de)
  print('\nPhoto Collage Created')
  
  if os.name == 'nt':
    os.system('start %windir%\explorer.exe "{}"'.format(os.path.abspath(video_thumb_folder)))
print('\r       Ready      ')

In [ ]:
video_path         = '/content/1.mp4'
video_thumb_folder = '123'
for_seconds        = 60

video_to_img(video_path, video_thumb_folder, for_seconds)

## Youtube Downloader

In [ ]:
!pip install youtube-dl
import youtube_dl
import json
import os
# ydl = youtube_dl.YoutubeDL({'outtmpl': '%(id)s.%(ext)s'})
# ydl = youtube_dl.YoutubeDL({'outtmpl': '%(title)s-%(id)s.%(ext)s'})

def thumbnail_downloader(url, img_name):
  !wget $url -O"$img_name" >/dev/null 2>&1
  # from IPython.display import Image
  # Image('1.jpg')

def save_json(result, file_name):
  video_detail = {}
  video_detail['Id']          = result['id']
  video_detail['Title']       = result['title']
  video_detail['Duration']    = result['duration']
  video_detail['Upload_date'] = result['upload_date']
  video_detail['Description'] = result['description']

  video_detail['Uploader']    = result['uploader']
  video_detail['Channel_url'] = result['channel_url']

  video_detail['Thumbnail']   = result['thumbnail']
  video_detail['Categories']  = result['categories']
  video_detail['Tags']        = result['tags']

  video_detail['View_count']  = result['view_count']
  video_detail['Like_count']  = result['like_count']
  video_detail['Dislike_count'] = result['dislike_count']
  video_detail['No_formats']  = len(result['formats'])
  video_detail['Z_Technical_details']     = result['formats'][-1]

  with open(file_name, 'w')as f1:
    f1.write(json.dumps(video_detail, indent=4, sort_keys=True))
  # print(json.dumps(video_detail, indent=4, sort_keys=True))

In [ ]:
def download_video(url, folder):
  if not os.path.exists(folder):
    os.makedirs(folder)

  ydl = youtube_dl.YoutubeDL({'outtmpl': '{}/%(title)s-%(id)s.%(ext)s'.format(folder)})
  with ydl:
    result = ydl.extract_info(url)
  
  if video_name == '':
    pass
  json_name  = video_name.replace('.mp4', '.json')
  thumb_name = video_name.replace('.mp4', '.jpg')

  thumbnail_downloader(result['thumbnails'][-1]['url'], thumb_name)
  save_json(result, json_name)

In [ ]:
download_video(url, folder)

In [ ]:
video_url = 'https://www.youtube.com/watch?v=WsBApPjUG4k'

with ydl:
  result = ydl.extract_info(video_url, download=False)

if 'entries' in result:
  # Can be a playlist or a list of videos
  video = result['entries'][0]
else:
  # Just a video
  video = result

print(video)
# video_url = video['url']
# print(video_url)
with open('1.json', 'w')as f1:
  f1.write(json.dumps(video, indent=4))

In [ ]:
with open('1.json', 'w')as f1:
  f1.write(json.dumps(result, indent=4, sort_keys=True))